<a href="https://colab.research.google.com/github/pstansfeld/MemProtMD/blob/main/gnina_ligand_docking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://avatars.githubusercontent.com/u/15654234?s=200&v=4" height="200" align="right" style="height:240px">

##Ligand Docking with Gnina

This Google Colab uses [Gnina](https://doi.org/10.1186/s13321-021-00522-2) to dock ligands to proteins.

Change the settings below and click **Runtime → Run all**. 

You will be prompted to upload a Protein PDB file of your Recetor and then a Ligand PDB to dock to the Receptor.

In [ ]:
#@title Initialisation
%%capture
import os
import sys
if os.path.isfile("/content/gnina") == False:
  !python3 -m pip install py3dmol
  !python3 -m pip install colorama
  !apt-get update
  !apt-get upgrade
  !apt-get install gzip
  !apt-get install openbabel
  !wget https://github.com/gnina/gnina/releases/download/v1.0.1/gnina
  !chmod +x gnina
  import py3Dmol
  from google.colab import files
  from colorama import Fore
  sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [ ]:
#@title Upload Receptor and Ligand files
os.chdir('/content/')

print(Fore.BLUE + "\nUpload Receptor Structure:\n")
upload = files.upload()
receptorname = next(iter(upload))

name = os.path.splitext(receptorname)[0]
working_dir = '/content/' + name + '/'
os.makedirs(working_dir, exist_ok=True)
os.rename(receptorname, working_dir + name + '.pdb')
os.chdir(working_dir)
mol1 = open(working_dir + receptorname, 'r').read()
mview = py3Dmol.view(width=800,height=400) 
mview.addModel(mol1,'pdb')
mview.setStyle({'cartoon':{'color':'spectrum'}})
mview.setBackgroundColor('0xffffff')
mview.zoomTo()
mview.show()

print(Fore.BLUE + "\nUpload Ligand Structure:\n")
upload = files.upload()
ligandname = next(iter(upload))

mol1 = open(working_dir + ligandname, 'r').read()
mview = py3Dmol.view(width=800,height=400) 
mview.addModel(mol1,'pdb')
mview.setStyle({'model':1},{'stick':{'colorscheme':'dimgrayCarbon','radius':.5}})
mview.setBackgroundColor('0xffffff')
mview.zoomTo()
mview.show()

In [ ]:
#@title Run Gnina & Show Dockings & Analyses
!/content/gnina -r $receptorname -l $ligandname -o docked.sdf --seed 0 --autobox_ligand $ligandname
v = py3Dmol.view()
v.addModel(open(receptorname).read())
v.setStyle({'cartoon':{}})
v.addModel(open(ligandname).read())
v.setStyle({'model':1},{'stick':{'colorscheme':'dimgrayCarbon','radius':.125}})
v.addModelsAsFrames(open('docked.sdf').read())
v.setStyle({'model':2},{'stick':{'colorscheme':'greenCarbon'}})
v.animate({'interval':1500})
v.zoomTo({'model':1})
!obrms -firstonly $ligandname docked.sdf

In [ ]:
#@title Download Zip
os.chdir('/content/')
os.system('zip -r ' + name + '.zip ' + name )
files.download(name + '.zip')